In [1]:
from myosuite.utils import gym

MyoSuite:> Registering Myo Envs


Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
/home/ubuntu/myochallenge_2025eval/.venv/lib/python3.11/site-packages/gymnasium/envs/registration.py:694: UserWarning: WARN: Overriding environment myoArmReachFixed-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/home/ubuntu/myochallenge_2025eval/.venv/lib/python3.11/site-packages/gymnasium/envs/registration.py:694: UserWarning: WARN: Overriding environment myoSarcArmReachFixed-v0 already in registry.
  logger.warn(f"Overriding environment {ne

In [23]:
import numpy as np

In [2]:
env = gym.make("myoChallengeTableTennisP1-v0", max_episode_steps=300)

/home/ubuntu/myochallenge_2025eval/.venv/lib/python3.11/site-packages/glfw/__init__.py:917: GLFWError: (65550) b'X11: The DISPLAY environment variable is missing'
  warnings.warn(message, GLFWError)


    MyoSuite: A contact-rich simulation suite for musculoskeletal motor control
        Vittorio Caggiano, Huawei Wang, Guillaume Durandau, Massimo Sartori, Vikash Kumar
        L4DC-2019 | https://sites.google.com/view/myosuite
    


In [4]:
env.reset()
action = env.action_space.sample()
obs, base_reward, terminated, truncated, info = env.step(action)

In [24]:
obs_dict = info['obs_dict']

In [28]:
time = obs_dict["time"] - 0

obs = np.concatenate([
    np.array([time], dtype=np.float32),

    obs_dict["pelvis_pos"],
    obs_dict["body_qpos"],
    obs_dict["body_qvel"],

    obs_dict["ball_pos"],
    obs_dict["ball_vel"],

    obs_dict["paddle_pos"],
    obs_dict["paddle_vel"],
    obs_dict["paddle_ori"],
    obs_dict["padde_ori_err"],

    obs_dict["reach_err"],
    obs_dict["palm_pos"],
    obs_dict["palm_err"],

    obs_dict["touching_info"],
    obs_dict["act"],
], axis=0)

In [26]:
obs

array([ 1.0000e-02,  2.0205e+00, -2.3418e-03,  9.5000e-01, -4.2046e-01,
        1.6721e-03, -5.3598e-03,  1.1681e-03, -3.6445e-03, -3.4621e-06,
        9.6579e-06, -3.3251e-03, -9.1532e-04,  2.3942e-04, -2.4460e-04,
       -8.7781e-02,  3.9781e-04, -1.8701e-04, -1.1165e-03,  4.4584e-04,
       -1.3686e-04, -1.3278e-03,  4.1474e-04, -1.0519e-04, -1.4927e-02,
        5.4844e-03, -5.4581e-03,  1.4797e-02, -4.0662e-03,  1.7998e-02,
        2.9289e-03, -2.8127e-03, -1.8155e-02,  1.2475e-01,  1.7207e-01,
        7.0758e-01, -6.9755e-01, -3.7125e-01,  7.9200e-01, -1.1381e-01,
       -1.0268e-01, -7.3674e-01,  1.2151e-01,  7.3510e-01,  7.2434e-02,
       -2.9483e-01,  6.8161e-01, -1.5828e-01,  3.1198e-01,  3.9699e-01,
        6.9206e-01, -1.4910e-01,  4.3916e-01,  5.1121e-01,  9.2356e-01,
       -1.8917e-01,  2.5825e-01,  5.0088e-01,  8.0952e-01, -1.9697e-01,
        2.3484e-01,  8.5903e-04,  7.6031e-03,  2.7301e-01, -7.6879e-01,
        1.9316e-01, -5.4072e-01, -1.0528e-03,  1.5417e-03, -4.80

In [11]:
info['rwd_dict']

OrderedDict([('reach_dist', array(0.0383)),
             ('palm_dist', array(0.6039)),
             ('paddle_quat', array(0.3711)),
             ('torso_up', 0.9735566785991789),
             ('act_reg', array(-0.0161)),
             ('sparse', False),
             ('solved', array(False)),
             ('done', array(0)),
             ('dense', array(3.3234))])